In [4]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np

import qiime2 as q2

%matplotlib inline

# location of this week's data and all the results produced by this notebook 
# - this should be a path relative to your working directory
data_dir = 'Data'

In [6]:
df_meta_ITS = pd.read_csv('Data/20250913_metadata_ITS.tsv', sep='\t')
df_meta_ITS.head()

,sample ID,label_barcode1,label_barcode2,created_date,amplicon,unique_id,sample_id,person-id,plate,plate_position,amplicon_conc,DNA_extraction_plate,project,project_2,sample_type,day,hand
0,366291_386-LP4-ITS-0386,GGAAGTGAAGTT,ATTTGCTTTGCC,16.10.2024 14:59,ITS,LP4-ITS-0386,1820-d21-1,1820.0,P5,B01,6.7035,DNA59,highschool,highschool,sourdough,21.0,NaN
1,366291_387-LP4-ITS-0387,GATCGGGTCGGA,GTGACCCTGTCA,16.10.2024 14:59,ITS,LP4-ITS-0387,1725-d7-1,1725.0,P5,C01,7.8515,DNA59,highschool,highschool,sourdough,7.0,NaN
2,366291_388-LP4-ITS-0388,GTGACCTGATGT,GAGAATGGAAAG,16.10.2024 14:59,ITS,LP4-ITS-0388,1806-d14-1,1806.0,P5,D01,10.1119,DNA59,highschool,highschool,sourdough,14.0,NaN
3,366291_389-LP4-ITS-0389,GCCTACCGCCTA,CAGTAGCGATAT,16.10.2024 14:59,ITS,LP4-ITS-0389,1723-d21-1,1723.0,P5,E01,5.0523,DNA59,highschool,highschool,sourdough,21.0,NaN
4,366291_390-LP4-ITS-0390,ATACGGGCGCAT,GCCAAGGATAGG,16.10.2024 14:59,ITS,LP4-ITS-0390,1706-d7-1,1706.0,P5,F01,6.1151,DNA59,highschool,highschool,sourdough,7.0,NaN


In [8]:
df_meta_details = pd.read_csv('Data/20250914_metadata_personal_environmental_sensory_details.tsv', sep='\t')
df_meta_details.head()

,person-id,start_time,completion_time,background,ethical_agreement,sd_bake_experience,sd_bake_last_time,yeast_bake_experience,yeast_bake_last_time,sd_stor_loc,...,animal_score_D28,chemical_score_D28,body_odour_score_D28,fruity_score_D28,maillard_score_D28,sour_score_D28,ocean_score_D28,earthy_score_D28,fermented_dairy_score_D28,nutty_score_D28
0,1701,6.18.24 23:34:47,6.18.24 23:37:55,sterile,I agree.,no,NaN,yes,4.18.2024,kitchen counter,...,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1702,6.18.24 23:37:40,6.18.24 23:42:32,sterile,I agree.,no,NaN,yes,05.01.2023,cupboard,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1703,6.18.24 15:24:39,6.18.24 15:28:40,sterile,I agree.,no,NaN,yes,NaN,kitchen counter,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0
3,1704,6.24.24 21:14:24,6.24.24 21:16:48,sterile,I agree.,no,NaN,no,NaN,kitchen counter,...,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1705,6.24.24 21:41:15,6.24.24 21:46:29,sterile,I agree.,no,NaN,yes,2.19.2024,kitchen counter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ich weiss noch nicht was man raus nehmen kann